In [6]:
import duckdb

In [17]:
connection = duckdb.connect(
    database='./database/berlinmod.db',
    config={'allow_unsigned_extensions' : 'true'}
)

connection.load_extension('~/duckdb/mobilityduck/build/release/extension/mobilityduck/mobilityduck.duckdb_extension')
connection.load_extension('spatial');

## Create table and load data from CSV

In [18]:
query = """
CREATE OR REPLACE TABLE trips (
    tripid INTEGER,
    vehicleid INTEGER,
    posx DOUBLE,
    posy DOUBLE,
    t TIMESTAMPTZ
);
"""

connection.sql(query)

In [19]:
query = "COPY trips FROM 'data/trips_200k.csv';"
connection.sql(query)

In [20]:
query = "SELECT COUNT(*) FROM trips;"
result = connection.sql(query).fetchone()
result

(200000,)

In [21]:
query = "SELECT * FROM trips LIMIT 10;"
result = connection.execute(query).df()
result

,tripid,vehicleid,posx,posy,t
0,1,1,4.457924,50.889049,2020-06-01 09:48:50.886000+00:00
1,1,1,4.457894,50.889028,2020-06-01 09:48:52.386000+00:00
2,1,1,4.457721,50.888909,2020-06-01 09:48:57.523414+00:00
3,1,1,4.457928,50.888690,2020-06-01 09:49:05.623414+00:00
4,1,1,4.457930,50.888689,2020-06-01 09:49:05.668849+00:00
5,1,1,4.457976,50.888640,2020-06-01 09:49:07.468849+00:00
6,1,1,4.457999,50.888616,2020-06-01 09:49:09.718849+00:00
7,1,1,4.458068,50.888543,2020-06-01 09:49:12.418849+00:00
8,1,1,4.458073,50.888538,2020-06-01 09:49:12.611699+00:00
9,1,1,4.458188,50.888416,2020-06-01 09:49:17.111699+00:00


## Test `TINT()`

In [22]:
query = "SELECT TINT(vehicleid, t)::VARCHAR from trips;"
result = connection.execute(query).df()
result.head()

,"CAST(tint(vehicleid, t) AS VARCHAR)"
0,1@2020-06-01 09:48:50.886+00
1,1@2020-06-01 09:48:52.386+00
2,1@2020-06-01 09:48:57.523414+00
3,1@2020-06-01 09:49:05.623414+00
4,1@2020-06-01 09:49:05.668849+00


## Close connection

In [23]:
connection.close()